In [ ]:
library(Seurat)
library(Matrix)
library(tidyverse)

In [ ]:
source("../../util/sc_preprocess.R")

In [ ]:
# The following RDS files are products from import_DGE.R
dge_base   <- readRDS("vFeb2021_Drop_original_rawdge.rds")
dge_to10x <- readRDS("vFeb2021_Drop_to_10x_rawdge.rds")
dge_todrop <- readRDS("vFeb2021_Drop_to_drop_rawdge.rds")
dge_to10x_keepUMI  <- readRDS("vFeb2021_Drop_to_10x_keepUMI_rawdge.rds")
dge_todrop_keepUMI <- readRDS("vFeb2021_Drop_to_drop_keepUMI_rawdge.rds")

dge_todrop_seq2seq <- readRDS("drop_to_drop_SEQ2SEQ.drop.rds")

In [ ]:
bclist_to10x  <- read.table("barcode_correspondence_data/Fig4/bclist_drop_to_10X_random.txt",header=T)
bclist_toDrop <- read.table("barcode_correspondence_data//Fig4/bclist_drop_to_drop_random.txt",header=T)
bclist_to10x_keepUMI  <- read.table("barcode_correspondence_data//Fig4/bclist_drop_to_10X_keepUMI.txt",header=T); colnames(bclist_to10x_keepUMI) <- c("segment1.cellID","exCellID")
bclist_toDrop_keepUMI <- read.table("barcode_correspondence_data//Fig4/bclist_drop_to_drop_keepUMI.txt",header=T)

In [ ]:
colnames(dge_to10x) <- gsub("-1","",colnames(dge_to10x))
colnames(dge_to10x_keepUMI) <- gsub("-1","",colnames(dge_to10x_keepUMI))

dge_to10x_new  <- refresh_dge(dge_to10x,bclist_to10x$segment1.cellID,bclist_to10x$exCellID)
dge_todrop_new <- refresh_dge(dge_todrop,bclist_toDrop$segment1.cellID,bclist_toDrop$exCellID)
dge_to10x_keepUMI_new  <- refresh_dge(dge_to10x_keepUMI,bclist_to10x_keepUMI$segment1.cellID,bclist_to10x_keepUMI$exCellID)
dge_todrop_keepUMI_new <- refresh_dge(dge_todrop_keepUMI,bclist_toDrop_keepUMI$segment1.cellID,bclist_toDrop_keepUMI$exCellID)

In [ ]:
dge_target_list <- list(dge_todrop_new,dge_to10x_keepUMI_new,dge_todrop_keepUMI_new,dge_todrop_seq2seq)

In [ ]:
use_cells <- get_cells_used(dge_base,dge_target_list,percent_mito = 20,min_features = 200)

In [ ]:
store_dir <- "seurat_rds/"

orig.intersect.seu <- readRDS(paste0(store_dir,"Feb2021_Drop_original_Seurat.rds"))

In [ ]:
dge_todrop<- dge_todrop_seq2seq[,use_cells]

In [ ]:
orig.intersect.seu <- orig.intersect.seu[,use_cells]

In [ ]:
todrop.intersect.seu <- CreateSeuratObject(dge_todrop)

In [ ]:
high_var_genes <- VariableFeatures(orig.intersect.seu)

In [ ]:
todrop.intersect.seu <- getVarGenes_untilPCA(todrop.intersect.seu,var_genes=high_var_genes,num_varFeatures = 5000)

In [ ]:
todrop.intersect.seu <- seurat_processing_UMAP(todrop.intersect.seu,dimlimit = 20,res = 0.6)

In [ ]:
store_dir <- "/home/ha5656/work/Interstellar_test/seurat_obj/"

In [ ]:
#Fig drawing
draw_umap(seu_base = orig.intersect.seu,
          seu_to10x = orig.intersect.seu,
          seu_todrop = todrop.intersect.seu,
          col_seed = 22,
          w=6,
          h=6,
          outdir = "figure_out/",
          outname = "Mar2022_processed_Drop_to_drop_SEQ2SEQ_intersectionAll")

In [ ]:
#Export CellCellDist
v.vln.todrop <- get_dist_pcaspace_scatter(orig.intersect.seu,
                          todrop.intersect.seu,
                          outdir = "figure_outdrop_SEQ2SEQ/",
                          outname = "Mar2022_processed_drop_to_drop_SEQ2SEQ_intersectionAll")